In [1]:
# sys, file and nav packages:
import datetime as dt
import json

# math packages:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF

# charting:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import seaborn as sns

import IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display

start_date = '2020-03-01'
end_date ='2021-05-31'

a_qty = 20

a_fail_rate = .5

use_fail = False

unit_label = 'p/100m'

# survey data:
dfx= pd.read_csv('resources/checked_sdata_eos_2020_21.csv')

dfBeaches = pd.read_csv("resources/beaches_with_land_use_rates.csv")
dfCodes = pd.read_csv("resources/codes_with_group_names_2015.csv")

# set the index of the beach data to location slug
dfBeaches.set_index('slug', inplace=True)

# set the index of to codes
dfCodes.set_index("code", inplace=True)

# code description map
code_d_map = dfCodes.description

# code material map
code_m_map = dfCodes.material

# Changes in Spearmans $\rho$ as buffer size changes

Spearmans ranked test for correlation is being used to test the association of the number of objects found on the beach with the type of land use within a defined radius of the location where the objects were found.

The national survey on land use is the source for the classification of land use types. For every 100m² of the national territory there is an attributed land use classification based on the analysis of aerial images.

In [2]:
# this is the aggregated survey data that is being used
fd = dfx.copy()
fd['loc_dates'] = list(zip(fd.location, fd["date"]))
fd['date'] = pd.to_datetime(fd["date"])

# get rid of microplastics
mcr = dfx[dfx.groupname == "micro plastics (< 5mm)"].code.unique()
micro = ["G112", *mcr]
fd = fd[~fd.code.isin(micro)].copy()

# there was a data entry error
fd.code = fd.code.replace('G207', 'G208')

# the location data for the surveys
dfb = dfBeaches.loc[fd.location.unique()].copy()

# manager dict for key statisitics and
t = {}

t.update({"nlakes":fd[fd.w_t == 'l'].water_name_slug.nunique(),
          "nrivers":fd[fd.w_t == 'r'].water_name_slug.unique(),
         "ncodes":fd.code.nunique(),
         "quantity":fd.quantity.sum()}
        )

In [3]:
# methods for mapping values and aggregating data

def aggregate_buffer_data(data, cols, bufferdata, label_keys, **kwargs):
    """Maps values of <bufferdata> to <data> using the values in data.index.
    Each location is assigined a value from the <buffer data>. If the
    location does not have any corresponding buffer data it is assigned 0.
    
    :param data: A data frame with the locations
        of interest as the index.
    :type data: pandas.core.frame.DataFrame
    :param cols: A list of the new column names from
        the buffer data
    :type cols: list, array of column names
    :param bufferdata: A dataframe with two columns
        one for the landuse name and the other for the
        cumulative sum of that feature from the buffer
    :type bufferdata: pandas.core.frame.DataFrame
    :param label_keys: A mapping of numerical categories to labels
    :type label_keys: dict
    :return: The orginal data frame with the new columns added
    """
    
    # make empty columns for the measured land use features
    for acol in cols:
        data[acol]= 0

    # assign the land use value to each location
    for beach in data.index:
        # for each category of land use
        for label in list(label_keys):
            # assign a value
            try:
                new_data = bufferdata[(bufferdata.location == beach)&(bufferdata.label == label)].AS18_27.values[0]
            except:
                new_data = 0
            # assign that value to the location data
            data.loc[beach, label] = new_data
    
    return data

# account for the area attributed to water by removing the
# the value of water features from land use total
def adjusted_land_use(data, col_keys):
    """Account for the area attributed to water by removing the
    the value of water features from land use total.
    
    :param data: A data frame with the column luse_total
    :type data: pandas.core.frame.DataFrame
    :param col_keys: The column names of all non water land
        use columns.
    :type cols: list, array of column names
    :return: The orginal data frame with the new columns added
    """
    # total land use
    data['luse_total'] = data.loc[:,list(col_keys)].sum(axis=1)

    # amount attributed to water
    data['water_value'] = data.loc[:, ['lakes','rivers']].sum(axis=1)

    # the adjsuted land use
    # remove the water value from land use stats
    data['adjusted_land_use'] = data.luse_total - data.water_value
    
    return data

# determine the ratio of each landuse feature to the adjusted total for each buffer
def account_for_adj_luse(data, col_keys):
    """Uses the adjusted land use total to determine
    the % total for each feature.
    
    :param data: A data frame with the defined land use columns
    :type data: pandas.core.frame.DataFrame
    :param col_keys: The column names of all non water land
        use columns.
    :type cols: list, array of column names
    :return: The data frame with the part of total of the
        adjsuted land use for each AS1827 category
    """
    for label in list(col_keys.values()):
        a_label = F"part_{label}"
        data[a_label] = data[label]/data['adjusted_land_use']
    return data

# map survey results to land use stats
def assign_luse_stat_to_survey_results(sdata, luse_data, som_cols):
    """Maps the land-use data for a location to all samples at that
    location.
    
    :param data: A data frame with the survey data of the locations
        of interest
    :type data: pandas.core.frame.DataFrame
    :param luse_data: A dataframe with the land use stats
        with the locations of interest as index.
    :type luse_data: pandas.core.frame.DataFrame
    :param som_cols: The label of the land use attributes
        that are being mapped
    :type som_cols: list or array    
    :return: The orginal data frame with the new columns added    
    """
    for a_beach in sdata.location.unique():
        for element in som_cols:
            sdata.loc[sdata.location == a_beach, element] = luse_data.loc[a_beach, element]
    return sdata

# apply method and check results
def check_hypothesis(this_data, some_codes, variables, method):
    """Checks the ranked correlation (or whichever method is being used)
    of < some_codes > to < variables >.
    
    :param this_data: A data frame with the survey data of the locations
        of interest
    :type data: pandas.core.frame.DataFrame
    :param som_codes: The label of the dependent variables being
        tested
    :type som_codes: list or array 
    :param variables: The label of the independent variables being
        tested
    :type variables: list or array 
    :param method: A scipy method that compares to arrays and returns two values.
    :type method: scipy.stats.spearmanr or similar       
    :return: A dictionary of values key=variable, val=correlation coeffcient or zero
        depending on the value of p.    
    """
    myresults = {}
    for i,code in enumerate(some_codes):
        data = this_data[this_data.code == code]
        code_results ={code:{}}
        for j, n in enumerate(variables):
            corr, a_p = method(data[n], data[unit_label])
            if a_p <= 0.05:
                code_results[code].update({n:corr})
            else:
                code_results[code].update({n:"X"})
        myresults.update(code_results)
    
    return myresults


def make_col_totals(luse,key_labels, cols, colname='1500', idx='id'):
    """Convenience function to aggregate and label landuse values
    
    :param luse: A data frame with the survey data of the locations
        of interest
    :type data: pandas.core.frame.DataFrame
    :param key_labels: The labels for the different land use categories
    :type key_labels: list or array 
    :param  cols: The columns that need to be totaled
    :type cols: list or array 
    :param colname: The column name for the totals.
    :type colname: str
    :param idx: The column for the value labels
    :type idx: str,
    :return: A data frame      
    """
    x = luse[cols].copy()
    
    x.loc[colname]= x.sum(numeric_only=True, axis=0)
    
    y = x.loc[colname]

    a_tot = pd.DataFrame(index=y.index, data=y)

    a_tot[idx] = a_tot.index.map(lambda x: key_labels[x])
    
    return a_tot 

def agguse_groups(data, group_parts):
    """Aggregates the landuse values from the buffer output
    
    :param data: The buffer output from QGIS
    :type data: pandas.core.frame.DataFrame
    :param group_parts: A dict of the labels from the buffer data stored
        in arrays ie.. {'buildings':[2,3,4,5,9],'ind':[1]}
    :type group_parts: dict
    :return: A dict with the same keys as group_parts but the
        the values are the sum of the values in the buffer data 
        that correspond to the contents of the array.
    
    """
    wiw = {}
    for a_group in group_parts.keys():
        part_groups = [data.loc[x] for x in group_parts[a_group]]
        this_g = sum(part_groups)
        wiw.update({a_group:this_g})     
    
    return wiw
def landuse_per_radius(s, key_labels=key_labels, cols_lu=cols_lu, colname=1500, idx="id"):
    b = make_col_totals(s,key_labels, cols_lu, colname=radius, idx='id').set_index("id", drop=True)
    yi= agguse_groups(b, group_parts)
    c = pd.DataFrame.from_dict(yi, orient='index').append(b.loc[['total','water_value', 'adjusted_land_use']])
    return c

# aggregate the groups
def aggregate_the_luse_groups(data, group_parts, as_1827_part):
    """Makes new columns and makes a subtotal column for the
    newly created groups. Aggregates the functional groups defined
    by land use.
        
    :param data: A data frame with the defined land use columns
    :type data: pandas.core.frame.DataFrame
    :param group_parts: A dictionary that maps names to a list
        of column values.
    :type group_parts: dict
    :param as_1827_part: The AS1827 keys to labels dict
    :type as_1827_part: dict
    return: The data frame with the % of total of the
        adjsuted land use for each functional category
    """
    
    for a_group in group_parts.keys():
        part_groups = [as_1827_part[x] for x in group_parts[a_group]]
        new_group = F"% to {a_group}"
        data[new_group] = data.loc[:,part_groups].sum(axis=1)
    
    return data

def aggregate_adjust_account(data, add_these_cols, bufferdata, label_keys={}):
    data = aggregate_buffer_data(data, add_these_cols, bufferdata, list(label_keys.values()))
    data = adjusted_land_use(data, list(label_keys.values()))
    data = account_for_adj_luse(data, label_keys)
    
    return data


def make_bufferdata(a_file_name, a_col="AS18_27", label_keys={}):
    # these are the land use results
    d = pd.read_csv(a_file_name)
    if "slug" in d.columns:
        d.rename(columns={"slug":"location"}, inplace=True)
    else:
        pass

    # the buffer results from qgis
    e = d[[a_col, "location"]].copy()

    # make a df with location slug, landuse label and land use value:
    for a_num in e[a_col].unique():
        e.loc[e[a_col]==a_num, "label"]=label_keys[a_num]

    # counting the number of points inside the buffer for each location and category
    bufferdata = e.groupby(['location','label'], as_index=False)[a_col].count()
    
    return bufferdata


def check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius=1500, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=[]):
    
    # apply the defined groups, adjust and account for
    # the value of the water feature in the total land use
    data = aggregate_adjust_account(data, add_these_cols, bufferdata, label_keys=label_keys)

    # define the % of total land use for each group
    s = aggregate_the_luse_groups(data, group_parts, as_1827_part)

    # tag each result with the radius
    s['dist'] = radius
    
    # assign the land use to the survey result
    fd_luse = assign_luse_stat_to_survey_results(fd, s,som_cols) 
    this_data = fd_luse[[unit_label,*som_cols, 'code']]
    
    # check rho for each value
    myresults = check_hypothesis(this_data, abundant_codes, som_cols, stats.spearmanr)
    a = pd.DataFrame.from_dict(myresults, orient='index')
    a.replace('X', 0, inplace=True)
    a["dist"] = radius
    
    return a, s

def check_condition(x, conditions, i):
    
    if list(set(x)&set(conditions[i])):
        data = conditions[i][0]        
    elif i == 0 and not list(set(x)&set(conditions[i])):
        data = "Others"    
    else:
        data = check_condition(x, conditions, i-1)
    return data

NameError: name 'key_labels' is not defined

### Definition of abundant_codes

Currently the `< abundant_codes >` are defined as any object that had a value greater than `< a_qty >` at any survey. That is one possible way to identify objects that accumulate, the fail rate may be another.

In [ ]:
# decide which method to use
agg_this = {"quantity":"sum", "p/100m":"sum", "fail":"sum"}
c_totals = fd.groupby("code", as_index=False).agg(agg_this)
c_totals["fr"] = c_totals["fail"]/fd.loc_date.nunique()

if use_fail:
    abundant_codes = c_totals[c_totals.fr > a_fail_rate]

else:
    abundant_codes = fd[fd.quantity > a_qty].code.unique()

# order of code index
code_totals = fd[fd.code.isin(abundant_codes)].groupby("code").quantity.sum()
code_order = code_totals.sort_values(ascending=False).index

t.update({"qMostCommon":fd[fd.code.isin(abundant_codes)].quantity.sum()})

# either way the object description needs to be added to the survey results:
for code in fd.code.unique():
    fd.loc[fd.code == code, "description"] = code_d_map.loc[code]

# the descriptions were corrected of typos
for replaceme in to_replace:
    fd.loc[fd.description == replaceme, "description"] = new_descriptions[replaceme]

## 1500 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_1500.csv", a_col="AS18_27", label_keys=label_keys)

radius = 1500

# the functional land use groups:
these_groups = list(group_parts.keys())

add_these_cols = bufferdata.label.unique()

a1500, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

a1500["quantity"] = a1500.index.map(lambda x: code_totals.loc[x])
a1500["material"] = a1500.index.map(lambda x: code_m_map.loc[x])
a1500.sort_values(by="quantity", inplace=True)

# use intersects
water_int = fd[fd.code.isin(abundant_codes)][["loc_date", "code", "pcs_m", "quantity", "intersects"]]
for each_code in abundant_codes:
    new_data = water_int[water_int.code == each_code][["intersects", "pcs_m"]]
    c, p = stats.spearmanr(new_data.pcs_m.values, new_data.intersects.values)
    if p <= 0.05:
        a1500.loc[each_code, "intersects"] = c
    else:
        a1500.loc[each_code, "intersects"] = 0

totals=[]
lnd_u = []
totals.append(landuse_per_radius(s))
lnd_u.append(s)

a1500["description"] = a1500.index.map(lambda x: code_d_map.loc[x])

col_order = [a1500.columns[-1], *a1500.columns[:-1]]
a1500 = a1500.reindex(code_order)
a1500[col_order].head()

### Results Spearmans Rho

In [ ]:
# [Correlation matrix plot]
new_column_names = {
    "% to buildings":"Buildings",
    "% to ind":"Industries",
    "% to trans":"Roads",
    "% to recreation":"Recreational",
    "% to agg":"Agriculture",
    "% to woods":"Forests",
    "% to water":"Rivers/canals",
    "% to unproductive":"Unproductive"
}
# a1500.set_index('description', inplace=True)
a1500.rename(columns=new_column_names, inplace=True)
use_these = list(new_column_names.values())
aplot = a1500[[*use_these, "description"]].round(2)
aplot.set_index("description", inplace=True, drop=True)
fig, ax= plt.subplots(figsize=(17,17))
sns.set(font_scale=1.2)

kwargs = dict(annot=True, fmt="g", cmap='coolwarm',
              annot_kws={"size":9.5},center =0, mask=aplot==0,
              square = True, linewidths=0.1,linecolor="lightgrey",
              cbar = False)

ax = sns.heatmap(aplot, ax=ax, **kwargs)

ax.set_ylabel('')
ax.yaxis.tick_left()

ax.xaxis.tick_top() 
ax.xaxis.set_label_position('top')
ax.set_xticklabels(use_these,rotation=90)
       
ax_divider = make_axes_locatable(ax)
axins = inset_axes(ax,
                   width="60%",  # width = 5% of parent_bbox width
                   height="1.5%",  # height : 50%
                   loc='lower center',
                   bbox_to_anchor=(0.03, -0.03, 1, 1),
                   bbox_transform=ax.transAxes, 
                   borderpad=0
                   )

fig.colorbar(ax.get_children()[0], cax = axins, orientation = 'horizontal', extendfrac='auto')
# plt.savefig('output/corr_matrix_1500.png', bbox_inches='tight')

plt.show()

### Covariance of explanatory variables

In [ ]:
rho = aplot.corr(method='spearman')
pval = aplot.corr(method=lambda x, y: stats.spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

#### Sum of the total number of objects with a correlation (positive or negative) collected under the different land use categories

In [ ]:
# the cumulative sum of the objects of interest under
# the different landuse categories
cum_sums_objs = [a1500.loc[a1500[x] != 0, "quantity"].sum() for x in use_these]
a_df=pd.DataFrame(index=use_these, data=cum_sums_objs, columns=["total"])
a_df

#### The % total of materials of the objects of interest with respect to the total number of objects collected.

In [ ]:
# the % total of materials of the objects of interest
# with respect to the total amount collected
a_total = a1500.quantity.sum()
material_df = pd.DataFrame(a1500.groupby("material").quantity.sum() / a_total).round(3)
material_df

#### The % total of the top 36 objects with respect to all objects collected.

In [ ]:
# the % total of the 36 objects included in the analysis
# with respect to the total number of objects collected
index = ['ratio of top 36 items over all items:','number of top 36 items:']
data = [f"{(round((t['qMostCommon']/t['quantity'])*100))}%","{:,}".format(round(a_total))]
q_and_p = pd.DataFrame(data=data, index=index, columns=["value"])
q_and_p

#### The % total of the top 20 objects with respect to all objects collected

In [ ]:
# the % total of the top 20 objects included in the analysis
# with respect to the total number of objects collected
dT20 =a1500.iloc[:20,:]
dT20 = dT20[['description', 'quantity']]

index = ['ratio of top 20 items over all items:','number of top 20 items:']
data = [f"{(round((dT20.quantity.sum()/t['quantity'])*100))}%","{:,}".format(round(dT20.quantity.sum()))]

q_and_p20 = pd.DataFrame(data=data, index=index, columns=["value"])
q_and_p20

### The cumulative totals of the top 20 objects, grouped by economic source

In [ ]:
# define the broad categories:
tobaco = ["Tobacco", "Smoking related"]
industry = ['Industry','Construction', 'Industrial', 'Manufacturing']
sanitary = ["Sanitary", "Personal hygiene", "Water treatment"]
packaging = ['Packaging (non-food)','Packaging films nonfood or unknown', 'Paper packaging']
food = ['Food and drinks','Foil wrappers, aluminum foil', 'Food and drinks', 'Food and drink']
fragments = ['Plastic fragments and pieces',
             'Plastic fragments angular <5mm',
             'Styrofoam < 5mm', 
             'Plastic fragments rounded <5mm',
             'Foamed  plastic <5mm',
             'Fragmented plastics']

conditions = [tobaco, industry, sanitary, packaging, food, fragments]

codes = dT20.index    
    
for each_code in codes:
    srcs = dfCodes.loc[each_code][["source", "source_two", "source_three", "description"]]
    
    a = check_condition(srcs.values, conditions, len(conditions)-1)
    dT20.loc[each_code, "Type"] = a

fig, ax = plt.subplots(figsize=(7,7))
colors = {'Industry': 'firebrick', 'Tobacco': 'darkslategrey', 'Food and drinks': 'navy', 'Plastic fragments and pieces':'lightgrey',
         'Others':'linen','Sanitary':'plum','Packaging (non-food)':'saddlebrown'}
# N = 3
# ind = np.arange(N)
width = 0.6

labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]

ax.barh(dT20.description, dT20.quantity, color=[colors[i] for i in dT20.Type], edgecolor='darkgrey')
ax.invert_yaxis()  
ax.set_ylabel('')
ax.set_xticks([0,1000,2000,3000,4000,5000,6000,7000,8000])
ax.set_xticklabels([0,'1,000','2,000','3,000','4,000','5,000','6,000','7,000','8,000'])

ax.set_xlabel('Total item count', fontsize=16, labelpad =15)
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.tick_params(labelcolor='k', labelsize=14, width=1)
ax.yaxis.grid(color='lightgray')
ax.xaxis.grid(color='lightgray')
ax.set_facecolor('white')

plt.legend(handles, labels, fontsize=13,facecolor='white')
for ha in ax.legend_.legendHandles:
    ha.set_edgecolor("darkgrey")

plt.grid(True)
ax.spines['top'].set_color('0.5')
ax.spines['right'].set_color('0.5')
ax.spines['bottom'].set_color('0.5')
ax.spines['left'].set_color('0.5')

# plt.savefig('C:/Users/schre086/figures/land_use_ch/top_20items.png', bbox_inches='tight')

plt.show()

#### Litter items of local origin

Items that have four or more positive associations with a land use category.

In [ ]:
cols_corr = ['Buildings', 'Industries', 'Roads', 'Recreational', 'Agriculture', 'Forests', 'Unproductive']

c_count = a1500.copy()

# count the total number of correlations rho < 0 or rho > 0 and p < .05
c_count['corr_count'] = c_count[cols_corr].apply(lambda x: (x!=0).sum(), axis=1)

# count the total number of positive correlations
c_count['pos_corr_count'] = c_count[cols_corr].apply(lambda x: (x>0).sum(), axis=1)

# limt only to items that have four or more correlations
DfLocalItems = c_count[c_count.corr_count >= 4]

# get the local total
local_total = DfLocalItems.quantity.sum()

# pretty print this to jupyter
bullets = [f"* {x}" for x in DfLocalItems.description.unique()]

b = "\n".join(bullets)

a_string = f"""
* The total quantity of items that have more than four correlations, likely to be locally littered: {"{:,}".format(local_total)}
* Proportion of items likely to be of local origin among top items: {int((DfLocalItems.quantity.sum()/t["qMostCommon"])*100)}%

There are {len(DfLocalItems.description.unique())} categories in that group:\n\n {b}'
"""
md(a_string)

#### Ubiquitous items

Items that have three or less positive associations with a land use category and are positively associated with river or stream intersections.

In [ ]:
DfUbiItems = c_count[(c_count.corr_count <= 3)&(c_count.intersects > 0)]
ub_total = DfUbiItems.quantity.sum()

bullets = [f"* {x}" for x in DfUbiItems.description.unique()]
b = "\n".join(bullets)

a_string = f"""
* The total quantity of items with less than three correlations and at least one correlation to intersects: {"{:,}".format(ub_total)}
* Proportion of items likely to be locally littered among top items: {int((DfUbiItems.quantity.sum()/t["qMostCommon"])*100)}%

There are {len(DfUbiItems.description.unique())} categories in that group:\n\n {b}'
"""
md(a_string)

#### Other items

Items that have three or less positive associations with a land use category and are NOT positively associated with river or stream intersections.

In [ ]:
DfOthers = c_count[(c_count.corr_count <= 3)&(c_count.intersects <= 0)]
other_total = DfOthers.quantity.sum()
bullets = [f"* {x}" for x in DfOthers.description.unique()]

b = "\n".join(bullets)

a_string = f"""
* The total quantity of items with less than two positive correlations: {"{:,}".format(other_total)}
* Proportion of items likely to be locally littered among top items: {int((DfOthers.quantity.sum()/t["qMostCommon"])*100)}%

There are {len(DfOthers.description.unique())} categories in that group:\n\n {b}'
"""
md(a_string)

In [ ]:
# checking if total proportion is 100%
print('total proportion of groups:',((DfOthers.quantity.sum()/t["qMostCommon"])*100) 
      + ((DfUbiItems.quantity.sum()/t["qMostCommon"])*100) 
      + ((DfLocalItems.quantity.sum()/t["qMostCommon"])*100))

## 2000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 2000

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_2000.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a2000, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a2000["quantity"] = a2000.index.map(lambda x: code_totals.loc[x])
a2000["material"] = a2000.index.map(lambda x: code_m_map.loc[x])
a2000.sort_values(by="quantity", inplace=True)

a2000["description"] = a2000.index.map(lambda x: code_d_map.loc[x])

col_order = [a2000.columns[-1], *a2000.columns[:-1]]
a2000 = a2000.reindex(code_order)
a2000[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

## 2500 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 2500

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_2500.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a2500, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a2500["quantity"] = a2500.index.map(lambda x: code_totals.loc[x])
a2500["material"] = a2500.index.map(lambda x: code_m_map.loc[x])
a2500.sort_values(by="quantity", inplace=True)

a2500["description"] = a2500.index.map(lambda x: code_d_map.loc[x])

col_order = [a2500.columns[-1], *a2500.columns[:-1]]
a2500 = a2500.reindex(code_order)
a2500[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

## 3000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 3000

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_3000.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a3000, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)
totals.append(landuse_per_radius(s))
lnd_u.append(s)

a3000["quantity"] = a3000.index.map(lambda x: code_totals.loc[x])
a3000["material"] = a3000.index.map(lambda x: code_m_map.loc[x])
a3000.sort_values(by="quantity", inplace=True)

a3000["description"] = a3000.index.map(lambda x: code_d_map.loc[x])

col_order = [a3000.columns[-1], *a3000.columns[:-1]]
a3000 = a3000.reindex(code_order)
a3000[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

## 3500 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 3500

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_3500.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a3500, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a3500["quantity"] = a3500.index.map(lambda x: code_totals.loc[x])
a3500["material"] = a3500.index.map(lambda x: code_m_map.loc[x])
a3500.sort_values(by="quantity", inplace=True)

# use intersects
water_int = fd[fd.code.isin(abundant_codes)][["loc_date", "code", "pcs_m", "quantity", "intersects"]]
for each_code in abundant_codes:
    new_data = water_int[water_int.code == each_code][["intersects", "pcs_m"]]
    c, p = stats.spearmanr(new_data.pcs_m.values, new_data.intersects.values)
    if p <= 0.05:
        a3500.loc[each_code, "intersects"] = c
    else:
        a3500.loc[each_code, "intersects"] = 0

a3500["description"] = a3500.index.map(lambda x: code_d_map.loc[x])

col_order = [a3500.columns[-1], *a3500.columns[:-1]]
a3500 = a3500.reindex(code_order)
a3500[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

### Results Spearmans Rho

In [ ]:
# [Correlation matrix plot]
new_column_names = {
    "% to buildings":"Buildings",
    "% to ind":"Industries",
    "% to trans":"Roads",
    "% to recreation":"Recreational",
    "% to agg":"Agriculture",
    "% to woods":"Forests",
    "% to water":"Rivers/canals",
    "% to unproductive":"Unproductive"
}
# a1500.set_index('description', inplace=True)
a3500.rename(columns=new_column_names, inplace=True)
use_these = list(new_column_names.values())
aplot = a3500[[*use_these, "description"]].round(2)
aplot.set_index("description", inplace=True, drop=True)

fig, ax= plt.subplots(figsize=(17,17))
sns.set(font_scale=1.2)

kwargs = dict(annot=True, fmt="g", cmap='coolwarm',
              annot_kws={"size":9.5},center =0, mask=aplot==0,
              square = True, linewidths=0.1,linecolor="lightgrey",
              cbar = False)

ax = sns.heatmap(aplot, ax = ax, **kwargs)

ax.set_ylabel('')
ax.yaxis.tick_left()

ax.xaxis.tick_top() 
ax.xaxis.set_label_position('top')
ax.set_xticklabels(use_these,rotation=90)
       
ax_divider = make_axes_locatable(ax)
axins = inset_axes(ax,
                   width="60%",  # width = 5% of parent_bbox width
                   height="1.5%",  # height : 50%
                   loc='lower center',
                   bbox_to_anchor=(0.03, -0.03, 1, 1),
                   bbox_transform=ax.transAxes, 
                   borderpad=0
                   )

fig.colorbar(ax.get_children()[0], cax = axins, orientation = 'horizontal', extendfrac='auto')
plt.show()

#### Litter items of local origin

Items that have four or more positive associations with a land use category.

In [ ]:
# cols_corr = ['Buildings', 'Industries', 'Roads', 'Recreational', 'Agriculture', 'Forests', 'Unproductive']

c_count = a3500.copy()

# count the total number of correlations rho < 0 or rho > 0 and p < .05
c_count['corr_count'] = c_count[cols_corr].apply(lambda x: (x!=0).sum(), axis=1)

# count the total number of positive correlations
c_count['pos_corr_count'] = c_count[cols_corr].apply(lambda x: (x>0).sum(), axis=1)

# limt only to items that have four or more correlations
DfLocalItems = c_count[c_count.corr_count >= 4]

# get the local total
local_total = DfLocalItems.quantity.sum()

# pretty print this to jupyter
bullets = [f"* {x}" for x in DfLocalItems.description.unique()]

b = "\n".join(bullets)

a_string = f"""
* The total quantity of items that have more than four correlations, likely to be locally littered: {"{:,}".format(local_total)}
* Proportion of items likely to be of local origin among top items: {int((DfLocalItems.quantity.sum()/t["qMostCommon"])*100)}%

There are {len(DfLocalItems.description.unique())} categories in that group:\n\n {b}'
"""
md(a_string)

#### Ubiquitous items

Items that have three or less positive associations with a land use category and are positively associated with river or stream intersections.

In [ ]:
DfUbiItems = c_count[(c_count.corr_count <= 3)&(c_count.intersects > 0)]
ub_total = DfUbiItems.quantity.sum()

bullets = [f"* {x}" for x in DfUbiItems.description.unique()]
b = "\n".join(bullets)

a_string = f"""
* The total quantity of items with less than three correlations and at least one correlation to intersects: {"{:,}".format(ub_total)}
* Proportion of items likely to be locally littered among top items: {int((DfUbiItems.quantity.sum()/t["qMostCommon"])*100)}%

There are {len(DfUbiItems.description.unique())} categories in that group:\n\n {b}'
"""
md(a_string)

#### Other items

Items that have three or less positive associations with a land use category and are NOT positively associated with river or stream intersections.

In [ ]:
DfOthers = c_count[(c_count.corr_count <= 2)&(c_count.intersects <= 0)]
other_total = DfOthers.quantity.sum()
bullets = [f"* {x}" for x in DfOthers.description.unique()]

b = "\n".join(bullets)

a_string = f"""
* The total quantity of items with less than two positive correlations: {"{:,}".format(other_total)}
* Proportion of items likely to be locally littered among top items: {int((DfOthers.quantity.sum()/t["qMostCommon"])*100)}%

There are {len(DfOthers.description.unique())} categories in that group:\n\n {b}'
"""
md(a_string)

In [ ]:
# checking if total proportion is 100%
print('total proportion of groups:',((DfOthers.quantity.sum()/t["qMostCommon"])*100) 
      + ((DfUbiItems.quantity.sum()/t["qMostCommon"])*100) 
      + ((DfLocalItems.quantity.sum()/t["qMostCommon"])*100))

### 4000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 4000

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_4000.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a4000, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a4000["quantity"] = a4000.index.map(lambda x: code_totals.loc[x])
a4000["material"] = a4000.index.map(lambda x: code_m_map.loc[x])
a4000.sort_values(by="quantity", inplace=True)

a4000["description"] = a4000.index.map(lambda x: code_d_map.loc[x])

col_order = [a4000.columns[-1], *a4000.columns[:-1]]
a4000 = a4000.reindex(code_order)
a4000[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

### 4500 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 4500

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_4500.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a4500, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a4500["quantity"] = a4500.index.map(lambda x: code_totals.loc[x])
a4500["material"] = a4500.index.map(lambda x: code_m_map.loc[x])
a4500.sort_values(by="quantity", inplace=True)

a4500["description"] = a4500.index.map(lambda x: code_d_map.loc[x])

col_order = [a4500.columns[-1], *a4500.columns[:-1]]
a4500 = a4500.reindex(code_order)
a4500[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

### 5000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 5000

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_5k.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a5000, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a5000["quantity"] = a5000.index.map(lambda x: code_totals.loc[x])
a5000["material"] = a5000.index.map(lambda x: code_m_map.loc[x])
a5000.sort_values(by="quantity", inplace=True)

a5000["description"] = a5000.index.map(lambda x: code_d_map.loc[x])

col_order = [a5000.columns[-1], *a5000.columns[:-1]]
a5000 = a5000.reindex(code_order)
a5000[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

### 10000 meters codes with an association to a landuse feature

Where p <= 0.05 for Spearmans test for association.

In [ ]:
# the locations that need land use data
data = pd.DataFrame(index = fd.location.unique())
radius = 10000

# the functional land use groups:
these_groups = list(group_parts.keys())

# the bufferdata for this radius
bufferdata = make_bufferdata("resources/buffer_output/luse_10k.csv", a_col="AS18_27", label_keys=label_keys)

add_these_cols = bufferdata.label.unique()

a10k, s = check_rho_for_this_radius(data, fd, add_these_cols, bufferdata, radius, label_keys=label_keys, group_parts=group_parts,
                              som_cols=som_cols, as_1827_part=as_1827_part, abundant_codes=abundant_codes)

totals.append(landuse_per_radius(s))
lnd_u.append(s)

a10k["quantity"] = a10k.index.map(lambda x: code_totals.loc[x])
a10k["material"] = a10k.index.map(lambda x: code_m_map.loc[x])
a10k.sort_values(by="quantity", inplace=True)

a10k["description"] = a10k.index.map(lambda x: code_d_map.loc[x])

col_order = [a10k.columns[-1], *a10k.columns[:-1]]
a10k = a10k.reindex(code_order)
a10k[col_order].sort_values(by=['% to buildings', '% to woods', '% to ind', '% to recreation', '% to agg'], ascending=True).head()

### Total correlations, total positive correlations and weight for each buffer radius

In [ ]:
# the total land use for each buffer zone
t_lu_by_r = pd.concat(totals, axis=1)

# the column names to aggregate
cols = list(new_column_names.values())

# these buffer values need the columns renamed
dfs_oldnames = [a2000, a2500, a3000, a4000, a4500, a5000, a10k]
dfs_newnames = [data.rename(columns=new_column_names) for data in dfs_oldnames]

# combined all the buffer values
dfs = [a1500,dfs_newnames[0], dfs_newnames[1], dfs_newnames[2], a3500, dfs_newnames[3], dfs_newnames[4], dfs_newnames[5], dfs_newnames[6]]

# count the total correlations and positive correlations
for data in dfs:
    data["count"] = data[cols].apply(lambda x: (x!=0).sum(), axis=1)
    data['positive'] = data[cols].apply(lambda x: (x>0).sum(), axis=1)

# convert meters to kilometers for index and printing
con_to_km = [x/1000 for x in t_lu_by_r.columns]

# for each radius get the total and positive count
total_correlation_count = {}
pos_correlations = {}
for i,a_radius in enumerate(con_to_km):
    
    data = dfs[i]
    
    total_count = data["count"].sum()
    pos_count = data["positive"].sum()
    label = a_radius
    total_correlation_count.update({label:total_count})
    pos_correlations.update({label:pos_count})
    
    
# combine the correlation totals per radius in to one df
t_corrs = pd.DataFrame.from_dict(total_correlation_count, orient="index", columns=["# of correlations"])

# add positive correlations
for k,v in pos_correlations.items():
    t_corrs.loc[k, "positive correlations"] = v

# add the weight factor
for i,a_radius in enumerate(con_to_km):
    data=dfs[i]    
    data['weight'] = (data['positive'] * data['quantity'])  
    t_corrs.loc[a_radius, "weight"] = data["weight"].sum()

t_corrs["positive correlations"] = t_corrs["positive correlations"].astype("int")
t_corrs["weight"] = t_corrs["weight"].astype("int")
t_corrs

## Changes in Spearmans $\rho$ at different buffer sizes

*Spearmans $\rho$ evaluated for expanded foams, fragmented plastics, snack wrappers, production pelltets and cigarettes at different buffer zone radii for % attributed to buildings*

*Table of results*

In [ ]:
comps = pd.concat(dfs)
comps.reset_index(inplace=True)
comps.rename(columns={'index':'code'}, inplace=True)

fig, ax = plt.subplots(figsize=(8,6))

data = comps[comps.code.isin(['G27', 'Gfrags', 'G30', 'Gfoam', 'G112'])].copy()
aval = "Buildings"
variables=[aval]
data[variables] = data[variables].astype('float').round(2)
data.sort_values(by=aval, inplace=True)

sns.lineplot(data=data, x='dist', y=aval, hue='code')
ax.set_ylabel('Rho', fontsize=14)
ax.set_xlabel('buffer radius meters', fontsize=14)
plt.legend(bbox_to_anchor=(1,1), loc='upper left')
plt.show()

## Changes of land use profile for different buffer zones

As the radius of the buffer zone changes the land use mix changes. Defining the radius of the buffer zone is determined by the scale at which the reporting is being done. For the report to Switzerland the target administrative level was the municipality. Therefore a radius of 1500m was appropriate, given the geographic size of a municipality in Switzerland.

*__Below:__ The percent total of land attributed to each category at each buffer radius* 

In [ ]:
# the row label for the total combined land use
total ='adjusted_land_use'
totaldf = t_lu_by_r.loc[total]

# divide the individual land use categories by the total land-use
# for each radius
landuse = ['agg', 'buildings', 'ind', 'recreation', 'trans', 'unproductive', 'woods']
adf = t_lu_by_r.loc[landuse].copy()
for element in totaldf.index:
    adf[element] =  ((adf[element]/totaldf.loc[element])*100).astype("int")

# pretty print
pprint =adf.applymap(lambda x:f"{x}%")
pprint

In [ ]:
data =adf.values
labels = adf.index
colors = adf.index
xlabels = [str(x) for x in adf.columns]

colors = ['bisque','lightcoral','k','orchid','lightgrey','saddlebrown', 'forestgreen']

bottom = [0]*(len(adf.columns))

width = 0.8      # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots(figsize=(8,5))

for i,group in enumerate(data):
    ax.bar(xlabels, group, width, bottom=bottom, label=labels[i], color = colors[i])
    bottom += group


ax.set_ylabel('Land-use profile [%]', fontsize=16)

ax.set_xlabel("Buffer zone radius [m]", labelpad =15, fontsize=16)
ax.set_facecolor('white')

ax.spines['top'].set_color('0.5')
ax.spines['right'].set_color('0.5')
ax.spines['bottom'].set_color('0.5')
ax.spines['left'].set_color('0.5')
ax.set_ylim(0,100)

ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.tick_params(labelcolor='k', labelsize=14, width=1)

ax.legend(bbox_to_anchor=(1,1), facecolor = 'white', fontsize=14)

plt.show()

*__Above:__ The percent total of land use at the different buffer radius. As the radius increases the % total attributed to forest and agriculture gets larger* 

In [ ]:
survey_data = dfx.copy()

In [ ]:
survey_data.info()

In [ ]:
# explanatory variables that are being considered
#luse_exp = ['% to buildings', '% to recreation', '% to agg', '% to woods', 'streets km', 'intersects']
luse_exp = ['Buildings [%]','Industrial [%]', 'Recreational [%]', 'Agricultural [%]', 'Forests [%]', 'Unproductive [%]', 'Road network \nlength [km]', 'Rivers/canals [#]']
# columns needed
use_these_cols = ['loc_date' ,
                  'date',
                  '% to buildings',
                  '% to trans',
                  '% to recreation',
                  '% to agg',
                  '% to woods',
                  '% to ind', 
                  '% to unproductive',
                  'population',
                  'water_name_slug',
                  'streets km',
                  'intersects',
                  'groupname',
                  'code'
                 ]

# the land use data was unvailable for these municipalities
no_land_use = ['Walenstadt', 'Weesen', 'Glarus Nord', 'Quarten']

# slice the data by start and end date, remove the locations with no land use data
use_these_args = ((survey_data.date >= start_date)&(survey_data.date <= end_date)&(~survey_data.city.isin(no_land_use)))
survey_data = survey_data[use_these_args].copy()

# format the data and column names
survey_data['date'] = pd.to_datetime(survey_data.date)

# work off a copy
new_som_data = survey_data.copy()

# adjust the fail rate for this aggregation of data
new_som_data['fail'] = True
new_som_data.loc[new_som_data.quantity == 0, 'fail'] = False

In [ ]:
# the survey total for each survey indifferent of object
dfdt = new_som_data.groupby(use_these_cols[:-2], as_index=False).agg({unit_label:'sum', 'quantity':'sum'})


dfdt = new_som_data.groupby(use_these_cols[:-2], as_index=False).agg({unit_label:'sum', 'quantity':'sum'})
dfdt.columns = ['loc_date','date','Built-up environment [%]', '% to tran', 'Recreational [%]', 'Agriculture [%]', 
                'Forests [%]', 'Industries [%]', 'Unproductive land [%]', 'Population', 'water_name_slug', 'Road network \nlength [km]', 'Rivers/canals [#]', 'p/100 m','quantity']

# method to get the ranked correlation of pcs_m to each explanatory variable
def make_plot_with_spearmans(data, ax, n):
    sns.scatterplot(data=data, x=n, y=unit_label, ax=ax, color='black', s=30, edgecolor='white', alpha=0.6)
    corr, a_p = stats.spearmanr(data[n], data[unit_label])
    return ax, corr, a_p

sns.set_style("whitegrid")
fig, axs = plt.subplots(1,4, figsize=(9,3), sharey=True)

data = dfdt.copy()

perc_cols = ['Built-up environment [%]','Industries [%]', 'Recreational [%]', 'Agriculture [%]', 'Forests [%]', 'Unproductive land [%]']

data[perc_cols] = data[perc_cols].apply(lambda x:x*100)

cols1 = ['Built-up environment [%]','Industries [%]', 'Recreational [%]', 'Agriculture [%]']
#cols2 = ['Forests [%]', 'Unproductive [%]', 'Road network \nlength [km]', 'Rivers/canals [#]']
for i, n in enumerate(cols1):
    ax=axs[i]
    
    # the ECDF of the land use variable
    the_data = ECDF(data[n].values)
    sns.lineplot(x=the_data.x, y= (the_data.y)*100, ax=ax, color='dodgerblue', label="% of surface area" )
    
    # get the median % of land use for each variable under consideration from the data
    the_median = data[n].median()
    
    # plot the median and drop horzontal and vertical lines
    ax.scatter([the_median], 50, color='red',s=50, linewidth=2, zorder=100, label="the median")
    ax.vlines(x=the_median, ymin=0, ymax=50, color='red', linewidth=2)
    ax.hlines(xmax=the_median, xmin=0, y=50, color='red', linewidth=2)
    
    #remove the legend from ax   
    ax.get_legend().remove()
    
    if i == 0:
        ax.set_ylabel("Share of \nsurveys [%]", labelpad = 15)
    else:
        pass
    
    # add the median value from all locations to the ax title
    ax.set_title(F"median: {(round(the_median, 2))}",fontsize=12, loc='left')
    ax.set_xlabel(n, fontsize=14, labelpad=15)

plt.tight_layout()

plt.savefig('C:/Users/schre086/figures/land_use_ch/land_use_1500_A.png', bbox_inches='tight')

plt.show()

In [ ]:
sns.set_style("whitegrid")
fig, axs = plt.subplots(1,4, figsize=(9,3), sharey=True)

#cols1 = ['Buildings [%]','Industrial [%]', 'Recreational [%]', 'Agricultural [%]']
cols2 = ['Forests [%]', 'Unproductive land [%]', 'Road network \nlength [km]', 'Rivers/canals [#]']
for i, n in enumerate(cols2):
    ax=axs[i]
    
    # the ECDF of the land use variable
    the_data = ECDF(data[n].values)
    sns.lineplot(x=the_data.x, y= (the_data.y)*100, ax=ax, color='dodgerblue', label="% of surface area" )
    
    # get the median % of land use for each variable under consideration from the data
    the_median = data[n].median()
    
    # plot the median and drop horzontal and vertical lines
    ax.scatter([the_median], 50, color='red',s=50, linewidth=2, zorder=100, label="the median")
    ax.vlines(x=the_median, ymin=0, ymax=50, color='red', linewidth=2)
    ax.hlines(xmax=the_median, xmin=0, y=50, color='red', linewidth=2)
    
    #remove the legend from ax   
    ax.get_legend().remove()
    
    if i == 0:
        ax.set_ylabel("Share of \nsurveys [%]", labelpad = 15)
    else:
        pass
    
    # add the median value from all locations to the ax title
    ax.set_title(F"median: {(round(the_median, 2))}",fontsize=12, loc='left')
    ax.set_xlabel(n, fontsize=14, labelpad=15)

plt.tight_layout()

plt.savefig('C:/Users/schre086/figures/land_use_ch/land_use_1500_B.png', bbox_inches='tight')

plt.show()

### sort this